```
Alexander Baquiax
12007988
```

In [140]:
import random
from typing import List, Tuple
import math

**Problema**

$\textrm{Max }  Z = \sum \limits _{i=1}^{N} V_i * X_i $
- $V_i$ representa la cantidad de azucar recolectable en la parcela $i$.
- $X_i$ representa una variable binaria que indica si se sembrará en la parcela $i$ o no.


**Restricciones**

$\sum \limits _{i=1}^{N} P_i * X_i \leq W$
- $P_i$ representa el tiempo estimado para cosechar en a parcela $i$.
- $X_i$ representa una variable binaria que indica si se sembrará en la parcela $i$ o no.
- $W$ representa el tiempo disponible para cosechar.

$\sum \limits _{i=1}^{N} V_i * X_i \leq C$
- $V_i$ representa la cantidad de azucar recolectable en la parcela $i$.
- $X_i$ representa una variable binaria que indica si se sembrará en la parcela $i$ o no.
- $C$ representa la cantidad máxima de carga

$X_i \in \{0,1\}$
- $X_i$ representa una variable binaria que indica si se sembrará en la parcela $i$ o no.

### Preparación

In [141]:
class Land:
    def __init__(self, id=0, capacity=100, cost=0.0):
        self.id: int = id
        self.capacity: float = capacity
        self.cost: float = cost

def build_lands(N = 100) -> Tuple[List[Land], float]: 
    lands: List[Land] = []
    lands_capacity: float = 0
    for i in range(1, N + 1):
        capacity = random.uniform(100.00, 1200.00)
        cost = random.uniform(1.0, 10.0)

        lands_capacity += capacity
        lands.append(Land(id=i, capacity=capacity, cost=cost))
        
    return lands, lands_capacity

In [142]:
def  build_pickups(min  = 50, max=100, min_capacity = 1000.00, max_capacity = 2500.00) -> float:
    total_capacity = 0.0
    for i in range(1, random.randint(min, max)):
        total_capacity += random.uniform(min_capacity, max_capacity)
    
    return total_capacity  


In [143]:
def build_people(min = 20, max = 100, hours_person=40) -> int:
    return random.randint(min, max) * hours_person

#### Inicialización de valores

In [144]:
N = 100 # total lands
lands, V = build_lands(N=100)
W = build_people(min=20, max=40, hours_person=40)
C = build_pickups(min=20, max=30, min_capacity=1000.00, max_capacity=2500.00)

print(f"Total lands: {len(lands)}")
print(f"Total lands capacity: {V:.2f} KG")
print(f"Total people time: {W:.2f} H")
print(f"Total pickups capacity: {C:.2f} KG")

Total lands: 100
Total lands capacity: 63131.46 KG
Total people time: 1040.00 H
Total pickups capacity: 38313.44 KG


#### Definición de funciones a usar


In [155]:
def objective(X: List[float] = []):
    return sum([x_i * v_i for x_i, v_i in zip(X, [land.capacity for land in lands])])

def restriction_a(X: List[float] = []):
    return sum([x_i * p_i for x_i, p_i in zip(X, [land.cost for land in lands])]) <= W

def restriction_b(X: List[float] = []):
    return sum([x_i * v_i for x_i, v_i in zip(X, [land.capacity for land in lands])]) <= C

In [146]:
PENALTY = 100

def fitness(X: List[float] = []):
    fit = objective(X)

    if not restriction_a(X):
        fit -= PENALTY

    if not restriction_b(X):
        fit -= PENALTY 
    
    return fit


### Algoritmo Genético

In [156]:
POPULATION_SIZE = 100
MUTATION_RATE = 0.1 

def genetic_algorithm():
    population: List[float] = [[random.choice([0.0, 1.0]) for _ in range(N)] for _ in range(100)]
    
    for _ in range(POPULATION_SIZE):
        fitness_values = [fitness(X) for X in population]
        
        ## Selection
        parents = random.choices(population, weights=fitness_values, k=POPULATION_SIZE)

        ## Crossover
        offspring = []
        for _ in range(POPULATION_SIZE):
            parent_a, parent_b = random.sample(parents, k=2)
            offspring.append([random.choice([a, b]) for a, b in zip(parent_a, parent_b)])

        ## Mutation
        for i in range(POPULATION_SIZE):
            for j in range(N):
                if random.random() < MUTATION_RATE:
                    offspring[i][j] = random.choice([0.0, 1.0])

        population = offspring

        best_fit = max(population, key=fitness)
    
    return best_fit

best_fit = genetic_algorithm()
print(f"Best solution with Z = {fitness(best_fit)} is {best_fit}" )

Best solution with Z = 42244.046480301346 is [1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0]


### Algoritmo de recocido

In [173]:
INITIAL_TEMPERATURE = 10000
FINAL_TEMPERATURE = 0.1
ALPHA = 0.9995

def simulated_annealing():
    current_solution = [random.choice([0.0, 1.0]) for _ in range(N)]
    current_value = objective(current_solution)
    best_solution = current_solution.copy()
    best_value = current_value  

    temperature = INITIAL_TEMPERATURE

    while temperature > FINAL_TEMPERATURE:
        next_solution = current_solution
        swap_index = random.randint(0, N - 1)

        next_solution[swap_index] = 1 - next_solution[swap_index]

        if not (restriction_a(next_solution) and restriction_b(next_solution)):
            temperature *= ALPHA
            continue

        next_value = objective(next_solution)

        if next_value > current_value or random.random() < math.exp((next_value - current_value) / temperature):
            current_solution = next_solution
            current_value = next_value

        if current_value > best_value:
            best_solution = current_solution.copy()
            best_value = current_value

        temperature *= ALPHA
        
    return best_solution, best_value

best_solution, best_value = simulated_annealing()
print(f"Best solution with Z = {best_value} is {best_solution}" )

Best solution with Z = 38311.576286724885 is [0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0]
